<a href="https://colab.research.google.com/github/carvedheart/youtube_summary/blob/main/video_url_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
!pip install pyspark pytubefix whisper-openai

from pytubefix import Playlist, Channel, YouTube
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import StringType
from pyspark import SparkConf
import pandas as pd
import whisper
import os
import re

In [158]:
def channel_or_playlist(url):
    """Xác định loại URL YouTube (Channel, Playlist hay Video thông thường)"""
    if "youtube.com" not in url and "youtu.be" not in url:
        return "Không phải URL YouTube"

    if not url.startswith(("http://", "https://")):
        url = "https://" + url

    from urllib.parse import urlparse, parse_qs
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)

    if "list" in query_params or "/playlist" in parsed_url.path:
        return "Playlist"

    if ("/channel/" in parsed_url.path or "/c/" in parsed_url.path or
        "/user/" in parsed_url.path or "/@" in parsed_url.path):
        return "Channel"

    return "Video thông thường hoặc không xác định"

In [159]:
def format_duration(seconds):
    """Chuyển đổi thời lượng từ giây sang định dạng h:m:s"""
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    if h:
        return f"{h}h {m}m {s}s"
    elif m:
        return f"{m}m {s}s"
    else:
        return f"{s}s"

def format_views(views):
    """Thêm dấu chấm cách mỗi 3 số"""
    return f"{views:,}".replace(",", ".")

def get_video_info(video):
    """Lấy thông tin cơ bản của một video YouTube, có định dạng đẹp"""
    try:
        return {
            "Title": video.title,
            "URL": video.watch_url,
            "Duration": format_duration(video.length),
            "Author": video.author,
            "Views": format_views(video.views)
        }
    except Exception as e:
        print(f"Lỗi khi xử lý video {getattr(video, 'watch_url', 'unknown')}: {str(e)}")
        return None

In [160]:
# 3. Hàm tóm tắt video (sử dụng Whisper)
def clean_subtitle(text):
    """Làm sạch phụ đề SRT bằng regex"""
    # Xóa dòng số thứ tự (1, 2, 3...)
    text = re.sub(r'^\d+\s*$', '', text, flags=re.MULTILINE)
    # Xóa dòng timestamp (00:00:00,000 --> 00:00:02,340)
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', text)
    # Xóa các tag HTML nếu có
    text = re.sub(r'<[^>]+>', '', text)
    # Xóa khoảng trắng thừa
    text = ' '.join(text.split())
    return text.strip()

def generate_summary(video_url):
    """Tạo tóm tắt video từ phụ đề hoặc audio (tối ưu)"""
    try:
        yt = YouTube(video_url)

        # Ưu tiên lấy phụ đề tiếng Anh/Việt
        caption = None
        for lang in ['en', 'vi', 'a.en', 'a.vi']:
            if lang in yt.captions:
                raw_text = yt.captions[lang].generate_srt_captions()
                caption = clean_subtitle(raw_text)
                break

        # Nếu không có phụ đề, dùng Whisper chuyển audio thành text
        if not caption:
            print(f"Đang xử lý audio cho video: {yt.title}")
            audio = yt.streams.filter(only_audio=True).first()
            audio_path = f"temp_{yt.video_id}.mp3"
            audio.download(filename=audio_path)

            model = whisper.load_model("base")
            result = model.transcribe(audio_path)
            os.remove(audio_path)
            caption = result["text"]

        # Tạo tóm tắt ngắn (30 từ đầu)
        summary = ' '.join(caption.split()[:30]) + '...'
        return summary

    except Exception as e:
        print(f"Lỗi khi tóm tắt video: {str(e)}")
        return None

In [161]:
# 4. Hàm xử lý chính với Spark (đã tích hợp summary)
def process_with_spark(url, spark):
    """Xử lý playlist/channel bằng PySpark với tính năng tóm tắt"""
    url_type = channel_or_playlist(url)

    if url_type == "Playlist":
        videos = Playlist(url).videos
        source_type = "Playlist"
    elif url_type == "Channel":
        videos = Channel(url).videos
        source_type = "Channel"
    else:
        return None

    # Tạo RDD từ danh sách video
    rdd = spark.sparkContext.parallelize(videos)

    def process_video(video):
        """Xử lý mỗi video bao gồm cả tóm tắt"""
        info = get_video_info(video)
        if info:
            summary = generate_summary(info["URL"])
            return {
                "Title": info["Title"],
                "URL": info["URL"],
                "Duration": info["Duration"],
                "Author": info["Author"],
                "Views": info["Views"],
                "Source_Type": source_type,
                "Summary": summary  # Đặt summary cuối cùng
            }
        return None

    # Ánh xạ và lọc kết quả
    video_info_rdd = rdd.map(process_video).filter(lambda x: x is not None)

    # Chuyển thành DataFrame
    return spark.createDataFrame(video_info_rdd)

In [162]:
def main():
    url = "https://www.youtube.com/playlist?list=PLOhREDBUkgUuYt8jfB7qEOJBzhcRNqg8I"

    # Cấu hình Spark tối ưu
    conf = SparkConf() \
        .set("spark.executor.memory", "4g") \
        .set("spark.driver.memory", "4g") \
        .set("spark.sql.shuffle.partitions", "8") \
        .set("spark.default.parallelism", "8") \
        .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

    spark = SparkSession.builder \
        .appName("YouTubeFastProcessing") \
        .config(conf=conf) \
        .getOrCreate()

    try:
        spark_df = process_with_spark(url, spark)

        if spark_df:

            # Hiển thị kết quả
            print("Thông tin video (bao gồm tóm tắt):")
            spark_df.show(truncate=False)
            # Lưu file nhanh hơn với coalesce
            spark_df.coalesce(1).write \
                .option("header", "true") \
                .mode("overwrite") \
                .csv("youtube_results")  # Dùng CSV thay vì Pandas

            print("Đã lưu kết quả vào thư mục youtube_results")

    finally:
        spark.stop()

In [163]:
if __name__ == "__main__":
    main()

Thông tin video (bao gồm tóm tắt):
+-------------+--------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------+---------------------------------------+---------+
|Author       |Duration|Source_Type|Summary                                                                                                                                                                  |Title                                               |URL                                    |Views    |
+-------------+--------+-----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------+---------------------------------------+---------+
|Chad Chad    |16m 52s |Playlist   